In [84]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import time
import hashlib
import hmac
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns


os.environ.pop("API_KEY")
os.environ.pop("SECRET_KEY")

load_dotenv()

API_KEY = os.environ["API_KEY"]
API_SECRET = os.environ["SECRET_KEY"]

# Helper Functions

In [135]:
def boilerplate(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    signature = hmac.new(
        API_SECRET.encode('utf-8'),
        query_string.encode('utf-8'),
        hashlib.sha256
    ).hexdigest()

    # Add the signature to the parameters
    params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")
    
def boilerplate1(params, endpoint):
    BASE_URL = 'https://api.binance.com'

    # Create the signature using HMAC-SHA256
    query_string = '&'.join([f"{key}={value}" for key, value in params.items()])
    # signature = hmac.new(
    #     API_SECRET.encode('utf-8'),
    #     query_string.encode('utf-8'),
    #     hashlib.sha256
    # ).hexdigest()

    # # Add the signature to the parameters
    # params['signature'] = signature

    # Define the headers with API key
    headers = {
        'X-MBX-APIKEY': API_KEY
    }

    # Send the GET request
    response = requests.get(BASE_URL + endpoint, headers=headers, params=params)

    # Check response status and print the result
    if response.status_code == 200:
        # print(response.json())
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return Exception(f"Error: {response.status_code}, {response.text}")

# Dual Investment Strategy
- **Put** Indicates that you are trying to buy the crypto product/ coin/ asset with stable coin. It means you are trying to buy Low
- **Call** Indicates that you are trying to Sell the crypto product/ coin/ asset for stable coin. It means you are trying to Sell High


In [86]:
def get_DCI_products(optionType, exercisedCoin, investCoin, pageSize, pageIndex):

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/list'

    # Define request parameters
    params = {
        'optionType': optionType,  #'CALL' or 'PUT'
        'exercisedCoin': exercisedCoin,  # Target exercised asset
        'investCoin': investCoin,  # Asset used for subscribing
        'pageSize': pageSize,  # Optional
        'pageIndex': pageIndex,  # Optional
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)


In [87]:
def get_DCI_account():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/accounts'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }

    return boilerplate(params, endpoint)



In [88]:
def get_DCI_positions():

    # Define the endpoint and base URL
    endpoint = '/sapi/v1/dci/product/positions'

    # Define request parameters
    params = {
        'recvWindow': 60000,  # Optional
        'timestamp': int(time.time() * 1000)  # Current timestamp in milliseconds
    }
    return boilerplate(params, endpoint)


In [89]:
def get_price_historical(symbol, interval):
    
    # Define the endpoint and base URL
    endpoint = '/api/v3/klines'

    # Define request parameters
    params = {
        'symbol': symbol,   
        'interval': interval  
    }
    return boilerplate1(params, endpoint)

# Runner


In [136]:
target = "ETH"
currency = "USDC"
target_currency = f"{target}{currency}"
print(target_currency)

ETHUSDC


In [137]:
data = get_price_historical(target_currency, "1d")
columns = [
    "Kline Open Time", "Open Price", "High Price", "Low Price", 
    "Close Price", "Volume", "Kline Close Time", "Quote Asset Volume", 
    "Number of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume"
]

cleaned_data = [row[:-1] for row in data]  # Remove the last element from each row
df_history = pd.DataFrame(cleaned_data, columns=columns)

df_history['Kline Open Time'] = pd.to_datetime(df_history['Kline Open Time'],unit='ms')
df_history['Kline Close Time'] = pd.to_datetime(df_history['Kline Close Time'],unit='ms')

df_history["Open Price"] = pd.to_numeric(df_history["Open Price"])
df_history["High Price"] = pd.to_numeric(df_history["High Price"])
df_history["Low Price"] = pd.to_numeric(df_history["Low Price"])
df_history["Close Price"] = pd.to_numeric(df_history["Close Price"])
df_history["Volume"] = pd.to_numeric(df_history["Volume"])
df_history["Quote Asset Volume"] = pd.to_numeric(df_history["Quote Asset Volume"])
df_history["Taker Buy Base Asset Volume"] = pd.to_numeric(df_history["Taker Buy Base Asset Volume"])
df_history["Taker Buy Quote Asset Volume"] = pd.to_numeric(df_history["Taker Buy Quote Asset Volume"])



df_copy = df_history

In [138]:
curr_price = df_history.tail(1)["Close Price"].iloc[0]
curr_price

np.float64(2524.1)

In [139]:
df_price_movements_column = []
for n in range(1, 31):
    df_price_movements_column.append(f'Price Movement {n} days')
    df_copy[f'Price Movement {n} days'] = (df_copy['Close Price'].shift(n) - df_copy['Close Price']) / df_copy['Close Price']


df_price_movements = df_copy.dropna()
df_price_movements=df_price_movements[df_price_movements_column]
df_price_movements


,Price Movement 1 days,Price Movement 2 days,Price Movement 3 days,Price Movement 4 days,Price Movement 5 days,Price Movement 6 days,Price Movement 7 days,Price Movement 8 days,Price Movement 9 days,Price Movement 10 days,...,Price Movement 21 days,Price Movement 22 days,Price Movement 23 days,Price Movement 24 days,Price Movement 25 days,Price Movement 26 days,Price Movement 27 days,Price Movement 28 days,Price Movement 29 days,Price Movement 30 days
30,-0.066443,-0.063135,-0.061903,-0.070852,-0.069465,-0.066667,-0.078808,-0.047053,-0.034129,-0.024517,...,-0.065879,-0.057678,-0.106094,-0.133663,-0.141839,-0.138482,-0.143699,-0.169204,-0.176562,-0.132426
31,0.033840,-0.034851,-0.031432,-0.030158,-0.039410,-0.037976,-0.035083,-0.047635,-0.014806,-0.001444,...,-0.023500,-0.034268,-0.025790,-0.075844,-0.104346,-0.112799,-0.109328,-0.114722,-0.141090,-0.148696
32,0.003841,0.037811,-0.031144,-0.027711,-0.026433,-0.035720,-0.034281,-0.031377,-0.043977,-0.011021,...,-0.028938,-0.019749,-0.030559,-0.022048,-0.072294,-0.100906,-0.109391,-0.105907,-0.111322,-0.137791
33,0.004488,0.008346,0.042469,-0.026796,-0.023348,-0.022064,-0.031393,-0.029947,-0.027030,-0.039687,...,-0.012376,-0.024580,-0.015350,-0.026208,-0.017659,-0.068131,-0.096871,-0.105394,-0.101894,-0.107334
34,0.005674,0.010187,0.014067,0.048384,-0.021274,-0.017807,-0.016515,-0.025897,-0.024443,-0.021509,...,-0.027596,-0.006772,-0.019046,-0.009764,-0.020683,-0.012085,-0.062843,-0.091747,-0.100318,-0.096798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,-0.035714,-0.038000,-0.051455,-0.049281,-0.050712,-0.042577,-0.101271,-0.098435,-0.111936,-0.131712,...,-0.032094,-0.025595,-0.018614,-0.041354,-0.060580,-0.033751,-0.036183,-0.060252,-0.048509,-0.067283
496,0.030174,-0.006617,-0.008973,-0.022833,-0.020594,-0.022068,-0.013688,-0.074153,-0.071231,-0.085140,...,-0.024064,-0.002888,0.003807,0.010998,-0.012428,-0.032234,-0.004595,-0.007101,-0.031896,-0.019799
497,0.016809,0.047491,0.010081,0.007686,-0.006408,-0.004131,-0.005630,0.002891,-0.058590,-0.055619,...,-0.066283,-0.007659,0.013872,0.020681,0.027992,0.004173,-0.015966,0.012137,0.009589,-0.015623
498,0.039391,0.056862,0.088752,0.049869,0.047379,0.032730,0.035097,0.033539,0.042396,-0.021507,...,-0.062781,-0.029503,0.031430,0.053809,0.060886,0.068486,0.043728,0.022796,0.052006,0.049357


In [140]:
summary_stats = df_price_movements.describe()
skewness = df_price_movements.skew()
kurtosis = df_price_movements.kurtosis()

In [141]:
# 1. Visualizing Price Movements Using Line Plot
fig_line = go.Figure()

# Create a line for each column
for col in df_price_movements.columns:
    fig_line.add_trace(go.Scatter(x=df_price_movements.index, y=df_price_movements[col], mode='lines', name=col))

# Update layout
fig_line.update_layout(
    title=f"Daily Price Movement Percentage {target}",
    xaxis_title="Time",
    yaxis_title="Percentage Movement",
    template='plotly'
)

# Show the line plot
fig_line.show()

# 2. Visualizing the Distribution with Box Plot
fig_box = px.box(df_price_movements, title=f"Distribution of Price Movements {target}")

# Show the box plot
fig_box.show()

# Test

In [362]:
target = "ETH"
currency = "USDC"
USDamt = 8499.48645

## Sell High
# AssetItem = target
# TargetItem = currency
# Direction = "CALL"

## Buy Low
TargetItem= target
AssetItem = currency
Direction = "PUT"

# Dual Investment tracking

In [363]:
first = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=1, pageSize=1)
full_list = first["list"]
total = first["total"]
print(total)

for i in range(total//100+1):
    try:
        curr = get_DCI_products(Direction, TargetItem, AssetItem, pageIndex=i+1, pageSize=100)
        full_list.extend(curr["list"])
    except Exception as e:
        print(f"ERROR found: {e}")

print("---------------------------")
print(len(full_list))

323
---------------------------
324


In [364]:
df = pd.DataFrame(full_list)
df["curr_price"]  = curr_price
df["strikePrice"] = pd.to_numeric(df["strikePrice"])

df["Percent_to_strikeprice"] = 100*(df["strikePrice"] - df["curr_price"])/df["curr_price"]
# df["percentage_over_"]

In [365]:
df = df.sort_values(by=['apr', 'duration'], ascending=[False, True])
df["strikePrice"] = df.strikePrice.astype(float)
df["apr"] = df.apr.astype(float)
df["1000return"] = df["apr"]*df["duration"]*1000/365
df["USDamt"] = df["1000return"]*USDamt/1000

df

,id,investCoin,exercisedCoin,strikePrice,duration,settleDate,purchaseDecimal,purchaseEndTime,canPurchase,apr,...,minAmount,maxAmount,createTimestamp,optionType,isAutoCompoundEnable,autoCompoundPlanList,curr_price,Percent_to_strikeprice,1000return,USDamt
0,1418201,USDC,ETH,2425.0,3,1730188800000,8,1730102400000,True,1.2733,...,0.1,4980864,1728650122000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-3.926152,10.465479,88.951201
1,1418201,USDC,ETH,2425.0,3,1730188800000,8,1730102400000,True,1.2733,...,0.1,4980864,1728650122000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-3.926152,10.465479,88.951201
2,1385385,USDC,ETH,2425.0,6,1730448000000,8,1730361600000,True,1.1225,...,0.1,4980864,1727748495000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-3.926152,18.452055,156.832990
3,1416296,USDC,ETH,2400.0,3,1730188800000,8,1730102400000,True,0.9521,...,0.1,4980864,1728481733000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-4.916604,7.825479,66.512557
4,1385424,USDC,ETH,2400.0,6,1730448000000,8,1730361600000,True,0.8998,...,0.1,4980864,1727748495000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-4.916604,14.791233,125.717883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1265689,USDC,ETH,1750.0,34,1732867200000,8,1732780800000,True,0.0369,...,0.1,1400000,1722844164000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-30.668357,3.437260,29.214947
315,1389742,USDC,ETH,1900.0,6,1730448000000,8,1730361600000,True,0.0368,...,0.1,4980864,1727765531000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-24.725645,0.604932,5.141607
316,1447129,USDC,ETH,1875.0,17,1731398400000,8,1731312000000,True,0.0368,...,0.1,1500000,1729570268000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-25.716097,1.713973,14.567887
317,1265607,USDC,ETH,1775.0,34,1732867200000,8,1732780800000,True,0.0368,...,0.1,1420000,1722844164000,PUT,True,"[STANDARD, ADVANCE]",2524.1,-29.677905,3.427945,29.135774


In [366]:
df.dtypes

id                         object
investCoin                 object
exercisedCoin              object
strikePrice               float64
duration                    int64
settleDate                  int64
purchaseDecimal             int64
purchaseEndTime             int64
canPurchase                  bool
apr                       float64
orderId                     int64
minAmount                  object
maxAmount                  object
createTimestamp             int64
optionType                 object
isAutoCompoundEnable         bool
autoCompoundPlanList       object
curr_price                float64
Percent_to_strikeprice    float64
1000return                float64
USDamt                    float64
dtype: object

In [367]:
fig = px.scatter(df, x="Percent_to_strikeprice", y="apr", hover_data=["id","orderId", "duration", "1000return"], title=f"{TargetItem} {Direction} target")
fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)
fig.show()


In [368]:
fig = px.scatter(df, x="strikePrice", y="apr", hover_data=["id","orderId", "duration", "1000return"], title=f"{TargetItem} {Direction} low target")
fig.update_layout(
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 0.1
    )
)
fig.show()


In [369]:
fig = px.scatter(df, x="duration", y="apr", hover_data=["strikePrice", "id", "1000return"])
fig.show()

In [370]:
fig = px.scatter(df, x="duration", y="1000return", color="strikePrice", hover_data=["id", "apr", "strikePrice"])
fig.show()

In [371]:
fig = px.scatter(df, x="strikePrice", y="1000return", hover_data=["id", "apr", "strikePrice", "duration"])
fig.show()

In [372]:
fig = px.scatter(df, x="strikePrice", y="apr", hover_data=["id", "apr", "strikePrice", "duration", "1000return"], color="1000return")
fig.show()

In [373]:
fig = px.line(df, x="strikePrice", y="apr", hover_data=["id", "apr", "strikePrice", "duration", "1000return"], color="duration")
fig.show()

In [374]:
# df_copy = df[df["strikePrice"]<=2320]

# df_copy = df_copy[df_copy["strikePrice"]>=2250]
# df_copy = df_copy[df_copy["duration"]<=25]
# # df_copy = df[df["duration"]>=0]

df_copy = df
fig = px.scatter(df_copy, x="strikePrice", y="duration",color="apr", hover_data=["id", "apr", "strikePrice", "duration", "1000return", "Percent_to_strikeprice"])



fig.show()



In [378]:
df_copy = df

# df_copy = df[df["strikePrice"]>=2200]
# df_copy = df_copy[df_copy["duration"]<=25]
# df_copy = df_copy[df_copy["apr"]>=0.2]

# df_copy = df_copy[df_copy["strikePrice"]<=60000 ]
# df_copy = df_copy[df_copy["strikePrice"]>=55000 ]

df_copy = df_copy[df_copy["strikePrice"]<=2500 ]
df_copy = df_copy[df_copy["strikePrice"]>=2150 ]


fig = px.scatter(df_copy, x="duration", y="apr", color='strikePrice', hover_data=["1000return", "USDamt", "Percent_to_strikeprice"])
fig.show()